코드 생성

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate
from mpl_toolkits.mplot3d import Axes3D
from collections import deque  
import random as rand

sats = [(1, 5), (2, 6), (3, 7), (4, 8), (0, 8), (1, 9), (0, 7), (1, 8), (2, 9), (1, 2),
            (2, 3), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (0, 3), (1, 4), (2, 5), (3, 6),
            (4, 7), (5, 8), (0, 2), (3, 5), (4, 6), (5, 7), (6, 8), (7, 9), (0, 5), (1, 6),
            (2, 7), (3, 8), (4, 9), (3, 9), (0, 6), (1, 7), (3, 9)]
g1tap = [2,9]
g2tap = [1,2,5,7,8,9]

def getCode(satsNum):
    
    g1 = deque(1 for i in range(10))
    g2 = deque(1 for i in range(10))
    
    # result
    g = []
    
    # Generating 1023 chips(One C/A sequence)
    for i in range(1023):
        val = (g1[9] + g2[satsNum[0]] + g2[satsNum[1]]) % 2
        g.append(val)
        
        #shift g1
        g1[9] = sum(g1[i] for i in g1tap) % 2
        g1.rotate()
        
        #shift g2
        g2[9] = sum(g2[i] for i in g2tap) % 2
        g2.rotate()
    # 0 => -1
    for n,i in enumerate(g):
            if i==0:
                g[n]=-1
        
    return g

# 코드 미리 만들어두기(시간복잡도 줄이려고)
codes = []
for i in range(37):
    codes.append(getCode(sats[i]))

In [3]:
OV = 1
ms = 100 # message 한 bit 마다 20ms

Data 생성

In [4]:
data = []
for i in range(ms//20):
    data.append((-1)**i)
RN = rand.randint(0,36)
t_code = codes[RN]
t_code = [chip for chip in t_code for _ in range(OV)]
t_code_inv = [x*-1 for x in t_code]
seq = []
for d in data:
    if d == 1:
        seq.extend(t_code*20)
    else:
        seq.extend(t_code_inv*20)



Signal 생성

In [5]:
def generate_transmitted_signal(code_delay, doppler_freq, svNumber, code_freq, signal_length, OV = 1.023e6*OV):
    delayed_seq = np.roll(seq, code_delay) # code_delay : delay된 sample 수
    
    t = np.arange(signal_length)
    code_phase = 2 * np.pi * code_freq * t / OV
    oversampled_signal = np.cos(code_phase) * np.exp(1j * (2 * np.pi * doppler_freq * t / OV))
    
    signal = delayed_seq * oversampled_signal[:signal_length]
    return signal
    

Replica 신호 생성

In [6]:
def generate_replica_signal(code_delay, doppler_freq, code_freq, svNumber, signal_length = 1023*OV ,OV = 1.023e6*OV ):
    ca_code = np.array(codes[svNumber])
    ca_code = [chip for chip in ca_code for _ in range(int(OV//1.023e6))]
    delayed_code = np.roll(ca_code, code_delay)
    
    t = np.arange(signal_length)
   
    code_phase = 2 * np.pi * code_freq * t / OV
    oversampled_signal = np.cos(code_phase) * np.exp(1j * (2 * np.pi * (doppler_freq) * t / OV))
    
    replica =  delayed_code * oversampled_signal[:signal_length]
    replica = np.tile(replica,20)
    return replica
    

Acquisition

In [7]:
def acquisition(signal, code_delay_range, doppler_freq_range,code_freq, svNumber, signal_length, OV):
    max_corr = 0
    max_code_delay = 0
    max_doppler_freq = 0
    max_cor_lst = []
    signal_div = [signal[i:i+(1023*20*OV)] for i in range(0,len(signal), 1023*20*OV)]
    
    for i, doppler_freq in enumerate(doppler_freq_range):
        replica = generate_replica_signal(code_delay_range[0],doppler_freq,code_freq, svNumber)
        
        for j, code_delay in enumerate(code_delay_range):
            correlation = []
            for lst in signal_div:
                cor = np.abs(correlate(lst, replica, mode='valid'))
                correlation.append(np.max(cor))
            avg_cor = sum(correlation)/len(signal_div)
            
            if avg_cor > max_corr:
                max_corr = avg_cor
                max_code_delay = code_delay
                max_doppler_freq = doppler_freq
                max_cor_lst = correlation
                
            replica = np.roll(replica, OV)
    return max_corr, max_code_delay, max_doppler_freq, max_cor_lst
            

Acquisition 실행

In [8]:
# Parameters
code_delay_range = np.arange(0, 200*OV, OV)  # Range of code delay in chips
doppler_freq_range = np.linspace(-5000, 5000, 21)  # Adjusted range of Doppler frequency in Hz
code_freq = 1.023e6  # Code frequency in Hz
svNumber = RN
# Generate received signal
true_code_delay = rand.randint(0,200*OV)
true_doppler_freq = rand.randint(-5000,5000)
received_signal = generate_transmitted_signal(true_code_delay, true_doppler_freq, svNumber, code_freq, 1023*20*(ms//20))

# Perform acquisition
max_corr, estimated_code_delay, estimated_doppler_freq, est_cor_lst= acquisition(received_signal, code_delay_range, doppler_freq_range, code_freq, svNumber,1023*20*(ms//20),OV)

print("-"*40)
print("Target SV Number:", svNumber + 1)
print("True Code Delay:", true_code_delay/OV)
print("True doppler frequency:", true_doppler_freq)
print("Maximum correlation:", max_corr)
print("Estimated code delay:", estimated_code_delay/OV)
print("Estimated Doppler frequency:", estimated_doppler_freq)
print('Est_cor_lst :',est_cor_lst)

----------------------------------------
Target SV Number: 21
True Code Delay: 21.0
True doppler frequency: -532
Maximum correlation: 921.2767174468911
Estimated code delay: 21.0
Estimated Doppler frequency: -500.0
Est_cor_lst : [890.7664217499499, 928.9042913711261, 928.904291371126, 928.9042913711257, 928.904291371127]


## Tracking loop test

fine frequency estimation

In [25]:
#input 재정의: 
#kLargest = doppler주파수estimation한걸로 넣
#data = received_signal로 넣ㅇㅇ
#GPS_fs = 샘플링 주파수 넣ㅇㅇ(아마?)


def GetFineFrequency(data, kLargest, GPS_fs): # now passed in data class
    # Performs fine-frequency estimation. In this case, data will be a slice
    # of data (probably same length of data that was used in the circular
    # cross-correlation)


    # Perform DFT on each of the ms of data (5 total), at kLargest frequency.
    # Uses variables from medium-frequency, so if they change, may need to re-create below.
    X = []
    PhaseAngle = []
    numMSmf = 1 # num ms for medium-frequency estimation
    Nmf = int(np.ceil(numMSmf*0.001*GPS_fs))  # num of samples to use for medium-frequency estimation (and DFT)
    # Create sampled time array for DFT
    Ts = 1/GPS_fs
    nTs = np.linspace(0,Ts*(Nmf + 1),Nmf,endpoint=False)
    
    for i in range(0,5):
        X.append(sum(data[i*1023:(i+1)*1023]*np.exp(-2*np.pi*1j*kLargest*nTs)))
        PhaseAngle.append(np.arctan(np.imag(X[i])/np.real(X[i])))
        print("Magnitude: %f" %X[i])
        print("Phase Angle: %f" %PhaseAngle[i])

    # Get difference angles
    PhaseDiff = []
    for i in range(1,5):
        PhaseDiff.append(PhaseAngle[i]-PhaseAngle[i-1])
        print("Phase difference %d, is: %f"%((i-1),PhaseDiff[i-1]))


    PhaseThreshold = (2.3*np.pi)/5
    for (i,curPhaseDiff) in enumerate(PhaseDiff):
        if np.abs(curPhaseDiff) > PhaseThreshold:
            curPhaseDiff = PhaseDiff[i] - 2*np.pi
            if np.abs(curPhaseDiff) > PhaseThreshold:
                curPhaseDiff = PhaseDiff[i] + 2*np.pi
                if np.abs(curPhaseDiff) > (2.2*np.pi)/5:
                    curPhaseDiff = PhaseDiff[i] - np.pi
                    if np.abs(curPhaseDiff) > PhaseThreshold:
                        curPhaseDiff = PhaseDiff[i] - 3*np.pi
                        if np.abs(curPhaseDiff) > PhaseThreshold:
                            curPhaseDiff = PhaseDiff[i] + np.pi
        PhaseDiff[i] = curPhaseDiff
    fList = (np.array(PhaseDiff)/(2*np.pi*0.001))
    print(fList)
    print()
    print(np.mean(fList))

    FineFrequencyEst = (np.mean(fList)) #각 데이터 슬라이스에서 계산된 위상 차이에 대한 주파수 변화율
    return FineFrequencyEst


In [29]:
kLargest = estimated_doppler_freq
GPS_fs= code_freq
data = received_signal
print(len(data))
FineFrequencyEst = GetFineFrequency(data, kLargest, GPS_fs)

print()
print(FineFrequencyEst + estimated_doppler_freq)
print("frequency 추정 완료")



102300
Magnitude: 0.996981
Phase Angle: 0.113781
Magnitude: -0.999649
Phase Angle: -0.087281
Magnitude: 0.962042
Phase Angle: -0.288343
Magnitude: -0.885674
Phase Angle: -0.489405
Magnitude: 0.773623
Phase Angle: -0.690467
Phase difference 0, is: -0.201062
Phase difference 1, is: -0.201062
Phase difference 2, is: -0.201062
Phase difference 3, is: -0.201062
[-32. -32. -32. -32.]

-31.999999999999275

-531.9999999999993
frequency 추정 완료


C:\Users\siljn\AppData\Local\Temp\ipykernel_14680\1322027041.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
  print("Magnitude: %f" %X[i])


In [ ]:



def GetFineFrequency(data, SatInfo, code5ms): # now passed in data class
    # Performs fine-frequency estimation. In this case, data will be a slice
    # of data (probably same length of data that was used in the circular
    # cross-correlation)

    
    """
    여기까지 doppler frequency 찾는 과정 -> 이미 수행한 작업
    Ts = 1/GPS_fs

    # Medium-frequency estimation data length (1ms in book, but may need to used
    # the data length from acquisition)
    numMSmf = 1 # num ms for medium-frequency estimation
    Nmf = int(np.ceil(numMSmf*0.001*GPS_fs))  # num of samples to use for medium-frequency estimation (and DFT)

    dataMF = data.CData[0:(4092*numMSmf)] #중간 주파수 추정에 사용될 데이터 선택

    # Create list of the three frequencies to test for medium-frequency estimation.
    k = []
    k.append(SatInfo.DopplerHz - 400*10**3) #여기서 추정한 doopplerHZ 받아와서 사용필요
    k.append(SatInfo.DopplerHz)
    k.append(SatInfo.DopplerHz + 400*10**3)

    # Create sampled time array for DFT - 우리거 샘플링된 시간 배열 다시 정의 필요 
    nTs = np.linspace(0,Ts*(Nmf + 1),Nmf,endpoint=False)

    # Perform DFT at each of the three frequencies.
    X = []
    X.append(np.abs(sum(dataMF*np.exp(-2*np.pi*1j*k[0]*nTs)))**2)
    X.append(np.abs(sum(dataMF*np.exp(-2*np.pi*1j*k[1]*nTs)))**2)
    X.append(np.abs(sum(dataMF*np.exp(-2*np.pi*1j*k[2]*nTs)))**2)

    # Store the frequency value that has the largest power
    kLargest = k[np.argmax(X)]
    print("Largest of three frequencies: %f"%kLargest) # Will remove. Temporarily for debugging purposes.
    """

    """
    #여기는 CA code랑 CW signal이랑 곱 
    # Get 5 ms of consecutive data, starting at beginning of CA Code
    CACodeBeginning = int(SatInfo.CodePhaseSamples)
    data5ms = data.CData[CACodeBeginning:int(5*4092) + CACodeBeginning]

    # Get 5 ms of CA Code, with no rotation performed.
    # passed in from function (code5ms)

    # Multiply data with ca code to get cw(continuous wave) signal
    dataCW = data5ms*code5ms
    """


    

    # Perform DFT on each of the ms of data (5 total), at kLargest frequency.
    # Uses variables from medium-frequency, so if they change, may need to re-create below.
    X = []
    PhaseAngle = []
    for i in range(0,5):
        X.append(sum(dataCW[i*4092:(i+1)*4092]*np.exp(-2*np.pi*1j*kLargest*nTs)))
        PhaseAngle.append(np.arctan(np.imag(X[i])/np.real(X[i])))
        print("Magnitude: %f" %X[i])
        print("Phase Angle: %f" %PhaseAngle[i])

    # Get difference angles
    PhaseDiff = []
    for i in range(1,5):
        PhaseDiff.append(PhaseAngle[i]-PhaseAngle[i-1])
        print("Phase difference %d, is: %f"%((i-1),PhaseDiff[i-1]))

    # Adjust phases so magnitude not greater than 2.3*pi/5
    # WIP
    PhaseThreshold = (2.3*np.pi)/5
    for (i,curPhaseDiff) in enumerate(PhaseDiff):
        if np.abs(curPhaseDiff) > PhaseThreshold:
            curPhaseDiff = PhaseDiff[i] - 2*np.pi
            if np.abs(curPhaseDiff) > PhaseThreshold:
                curPhaseDiff = PhaseDiff[i] + 2*np.pi
                if np.abs(curPhaseDiff) > (2.2*np.pi)/5:
                    curPhaseDiff = PhaseDiff[i] - np.pi
                    if np.abs(curPhaseDiff) > PhaseThreshold:
                        curPhaseDiff = PhaseDiff[i] - 3*np.pi
                        if np.abs(curPhaseDiff) > PhaseThreshold:
                            curPhaseDiff = PhaseDiff[i] + np.pi
        PhaseDiff[i] = curPhaseDiff
    fList = (np.array(PhaseDiff)/(2*np.pi*0.001))
    print(fList)
    print(np.mean(fList))

    FineFrequencyEst = 0 # Just a placeholder.
    return FineFrequencyEst
